In [ ]:
# https://youtu.be/9GzfUzJeyi0

"""
@author: Sreenivas Bhattiprolu
"""

import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import torch
import torch.backends.cudnn as cudnn

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization
#from keras.layers.normalization import BatchNormalization
import os
import seaborn as sns
from tensorflow.keras.applications.vgg16 import VGG16
import pickle


In [ ]:


total_classes=0  #to change when mee lay want to add more cows
#images_per_validation=5


#print(os.listdir("../Gold"))

SIZE = 224

import csv

#data = pd.read_csv('D:\\LamenessData\\September_6\\evening.csv',index_col ='Local_ID')

#COW_MAPPER = [list(row) for row in data.values]

train_images = []
train_labels = [] 
for directory_path in glob.glob("D:\LamenessData\September_6\DataByGroup\\AllCows\\Training/*"):
    label = directory_path.split("\\")[-1]
    #print(label)
    total_classes+=1
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        
        
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

print(total_classes)




In [ ]:

# test
test_images = []
test_labels = [] 
images_per_validation =[]
for directory_path in glob.glob("D:\LamenessData\September_6\DataByGroup\\AllCows\\Validation/*"):  #to change when baby add new images
    fruit_label = directory_path.split("\\")[-1]
    count=0
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)
        count+=1
    images_per_validation.append(count)
        
test_images = np.array(test_images)
test_labels = np.array(test_labels)
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)
print("Train lables encoded is here")
#print(train_labels_encoded)
#Split data into test and train datasets (already split but assigning to meaningful convention)




In [ ]:

x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

###################################################################
# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

#One hot encode y values for neural network. 
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)



In [ ]:


#############################
#Load model wothout classifier/fully connected layers
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in VGG_model.layers:
	layer.trainable = False
    
VGG_model.summary()  #Trainable parameters will be 0


#Now, let us use features from convolutional network for RF
feature_extractor=VGG_model.predict(x_train)

features = feature_extractor.reshape(feature_extractor.shape[0], -1)


X_for_RF = features #This is our X input to RF

print("generated featured")

#############################


In [ ]:

ytrain_name_filename = 'May_Weights/Y_TRAIN_V1.pkl'
features_filename = 'May_Weights/GENERAL_FEATURES_v1.pkl'
y_train = pickle.load(open(ytrain_name_filename, 'rb'))
features = pickle.load(open(features_filename, 'rb'))
#le = pickle.load(open(le_filename, 'rb'))


In [ ]:
import pickle
x_train = pickle.load(open('../June_Weights_HOG/GENERAL_X_TRAIN.pkl','rb'))
y_train = pickle.load(open('../June_Weights_HOG/GENERAL_Y_TRAIN.pkl','rb'))

In [ ]:
import lightgbm as lgb

lgb_train = lgb.Dataset(x_train, y_train)

# Set LightGBM parameters
params = {
    'objective': 'multiclass',
    'num_class': 146,  # Replace 'num_classes' with the actual number of classes in your dataset
    'metric': 'multi_logloss',
}

# Train the LightGBM model


In [ ]:
model = lgb.train(params, lgb_train, num_boost_round=100)
print('done')

In [ ]:


model_name ='../June_Weights_HOG/HOG_LIGHTGBM.pkl'
pickle.dump(model,open(model_name,'wb'))

print("done")


#########################



In [ ]:
VGG_model.save('May_Weights/GENERAL_VGG_v1.h5')


le_filename = 'May_Weights/GENERAL_LABEL_v1.le'
pickle.dump(le, open(le_filename, 'wb'))


In [ ]:
features_filename = 'May_Weights/GENERAL_FEATURES_v1.pkl'
pickle.dump(features, open(features_filename, 'wb'))

In [ ]:
#load modelsavedModels
#vgg_filename = 'vgg16_model.sav'
#svm_filename ='svm.sav'
#VGG_model = pickle.load(open(vgg_filename, 'rb'))
#clf = pickle.load(open(svm_filename, 'rb'))

In [ ]:
#run second part here

In [ ]:
#cv2.destroyAllWindows()

In [ ]:
#ipkernel_launcher.py --source "D:\Python\env\Lameness\Frames\Videos\20220201_145508_7108.mp4"  --yolo-weights weights_slm/best_6_23_gpu.pt --view-img --save-crop --device 0

In [ ]:
#python -h

In [ ]:
#print(X_For_RF) ## print me